# timesfm

In [5]:
import timesfm
import pandas as pd
import inspect

def get_library_attributes(lib_module):
    """
    ライブラリからパブリックな属性（関数、クラスなど）を取得し、DataFrameに格納する関数。
    
    Args:
        lib_module (module): 調査対象のモジュール（ここではtimesfm）
        
    Returns:
        pd.DataFrame: 名前、タイプ、ドキュメントを含むDataFrame
    """
    attributes_data = []

    # inspect.getmembersでライブラリ内の全メンバーを取得
    for name, obj in inspect.getmembers(lib_module):
        # プライベート属性（_で始まるもの）は除外する（通常は内部利用のため）
        if name.startswith("_"):
            continue

        # オブジェクトのタイプを判定
        if inspect.isclass(obj):
            obj_type = "Class (クラス)"
        elif inspect.isfunction(obj):
            obj_type = "Function (関数)"
        elif inspect.ismodule(obj):
            obj_type = "Module (モジュール)"
        else:
            obj_type = "Variable/Other (変数/その他)"

        # Docstring（ドキュメンテーション文字列：説明文）を取得
        doc = inspect.getdoc(obj)
        # ドキュメントがある場合、最初の1行のみを要約として取得（改行コードを除去）
        doc_summary = doc.split('\n')[0] if doc else "No description available"

        # リストに追加
        attributes_data.append({
            "Name": name,
            "Type": obj_type,
            "Docstring_Summary": doc_summary
        })

    # DataFrameを作成
    df = pd.DataFrame(attributes_data)
    
    return df

# メイン処理の実行
if __name__ == "__main__":
    try:
        # timesfmの情報を取得
        df_timesfm = get_library_attributes(timesfm)

        # 結果の表示（全行表示の設定をしておく）
        pd.set_option('display.max_colwidth', None)
        pd.set_option('display.max_rows', None)
        
        print("=== timesfm Library Structure ===")
        print(df_timesfm)

        # 必要に応じてCSV等に出力可能
        # df_timesfm.to_csv("timesfm_structure.csv", index=False)
        
    except ImportError as e:
        print(f"エラー: ライブラリのインポートに失敗しました。\n{e}")
    except Exception as e:
        print(f"予期せぬエラーが発生しました。\n{e}")

=== timesfm Library Structure ===
                     Name            Type  \
0          ForecastConfig     Class (クラス)   
1  TimesFM_2p5_200M_torch     Class (クラス)   
2                 configs  Module (モジュール)   
3             timesfm_2p5  Module (モジュール)   
4       timesfm_2p5_torch  Module (モジュール)   
5                   torch  Module (モジュール)   

                                             Docstring_Summary  
0                                     Options for forecasting.  
1  PyTorch implementation of TimesFM 2.5 with 200M parameters.  
2                         Abstract configs for TimesFM layers.  
3                                     No description available  
4                                              TimesFM models.  
5                                     No description available  


In [4]:
pip install git+https://github.com/google-research/timesfm.git

  Cloning https://github.com/google-research/timesfm.git to c:\users\hashimoto.ryohei\appdata\local\temp\pip-req-build-m2zp6xks
  Resolved https://github.com/google-research/timesfm.git to commit 2dcc66fbfe2155adba1af66aa4d564a0ee52f61e
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for timesfm: filename=timesfm-2.0.0-py3-none-any.whl size=43839 sha256=7e7044b7b37c38b3ed00bf9970840eabb235a1d21e36d3d17ccb3f836c98080b
  Stored in directory: C:\Users\hashimoto.ryohei\AppData\Local\Temp\pip-ephem-wheel-cache-flvnuovf\wheels\15\82\86\25cb4996f164a9e58ff452725de32b56291a099a808d2b4c14
Successfully built timesfm

   ------------- -------------------------- 1/3 [InquirerPy]
   -----------------------

  Running command git clone --filter=blob:none --quiet https://github.com/google-research/timesfm.git 'C:\Users\hashimoto.ryohei\AppData\Local\Temp\pip-req-build-m2zp6xks'

[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# /absolute/path は環境ごとに異なるので、出力は Path.resolve() で「フルパス化」します。
from __future__ import annotations

import importlib
import inspect
import pkgutil
from pathlib import Path
from typing import Any, Dict, List, Optional

import pandas as pd
import timesfm


def _safe_signature(obj: Any) -> Optional[str]:
    """関数/クラス等のシグネチャ(signature=引数の形)を安全に文字列化"""
    try:
        return str(inspect.signature(obj))
    except Exception:
        return None


def _safe_doc(obj: Any, max_chars: int = 800) -> Optional[str]:
    """docstring(説明文)を安全に取得して、長すぎる場合は切る"""
    try:
        doc = inspect.getdoc(obj) or ""
        doc = doc.strip()
        if not doc:
            return None
        return doc[:max_chars]
    except Exception:
        return None


def _iter_public_members(module: Any):
    """モジュールの公開メンバー（先頭が_でない）を列挙"""
    for name in dir(module):
        if name.startswith("_"):
            continue
        try:
            yield name, getattr(module, name)
        except Exception:
            continue


def collect_timesfm_api_catalog(
    include_methods: bool = True,
    max_doc_chars: int = 800,
) -> pd.DataFrame:
    """
    timesfm配下のサブモジュールを探索し、
    公開関数/クラス/メソッド等を DataFrame にまとめる。

    include_methods=True で class の公開メソッドも展開。
    """
    rows: List[Dict[str, Any]] = []

    pkg_name = timesfm.__name__
    pkg_paths = list(getattr(timesfm, "__path__", []))  # パッケージの探索パス

    # 1) まずルートモジュール自身
    modules: List[tuple[str, Any]] = [(pkg_name, timesfm)]

    # 2) サブモジュール発見 → import（失敗も記録）
    for m in pkgutil.walk_packages(pkg_paths, prefix=pkg_name + "."):
        modname = m.name
        try:
            mod = importlib.import_module(modname)
            modules.append((modname, mod))
        except Exception as e:
            rows.append(
                {
                    "module": modname,
                    "name": None,
                    "kind": "import_error",
                    "signature": None,
                    "doc": str(e),
                    "file": None,
                }
            )

    # 3) 各モジュールから公開メンバーを収集
    for modname, mod in modules:
        mod_file = getattr(mod, "__file__", None)

        for name, obj in _iter_public_members(mod):
            # 種別判定
            if inspect.isclass(obj):
                kind = "class"
            elif inspect.isfunction(obj) or inspect.isbuiltin(obj):
                kind = "function"
            elif inspect.ismodule(obj):
                kind = "module"
            elif callable(obj):
                kind = "callable"
            else:
                kind = "other"

            rows.append(
                {
                    "module": modname,
                    "name": name,
                    "kind": kind,
                    "signature": _safe_signature(obj) if kind in {"class", "function", "callable"} else None,
                    "doc": _safe_doc(obj, max_chars=max_doc_chars),
                    "file": mod_file,
                }
            )

            # 4) クラスなら公開メソッドも展開（多くなるのでスイッチ式）
            if include_methods and inspect.isclass(obj):
                try:
                    for meth_name, meth_obj in inspect.getmembers(obj):
                        if meth_name.startswith("_"):
                            continue
                        if not callable(meth_obj):
                            continue

                        rows.append(
                            {
                                "module": modname,
                                "name": f"{name}.{meth_name}",
                                "kind": "method",
                                "signature": _safe_signature(meth_obj),
                                "doc": _safe_doc(meth_obj, max_chars=max_doc_chars),
                                "file": mod_file,
                            }
                        )
                except Exception:
                    # クラスによっては getmembers が不安定なことがあるので握りつぶす
                    pass

    # 5) DataFrame化・整形
    df = pd.DataFrame(rows)
    df = df.drop_duplicates(subset=["module", "name", "kind"], keep="first")
    df = df.sort_values(["module", "kind", "name"], na_position="last").reset_index(drop=True)
    return df


if __name__ == "__main__":
    df = collect_timesfm_api_catalog(include_methods=True, max_doc_chars=800)

    # どんな「機能」があるかを見る例（関数・クラスだけ抽出）
    api_df = df[df["kind"].isin(["function", "class", "callable", "method"])].copy()

    # CSVに吐く（フルパスで保存先を表示）
    out_path = (Path.cwd() / "timesfm_api_catalog.csv").resolve()
    api_df.to_csv(out_path, index=False, encoding="utf-8-sig")

    print(api_df.head(20))
    print(f"\nSaved catalog to: {out_path}\nRows: {len(api_df)}")


                 module                                             name  \
0               timesfm                                   ForecastConfig   
1               timesfm                           TimesFM_2p5_200M_torch   
2               timesfm                   TimesFM_2p5_200M_torch.compile   
3               timesfm                  TimesFM_2p5_200M_torch.forecast   
4               timesfm  TimesFM_2p5_200M_torch.forecast_with_covariates   
5               timesfm           TimesFM_2p5_200M_torch.from_pretrained   
6               timesfm       TimesFM_2p5_200M_torch.generate_model_card   
7               timesfm           TimesFM_2p5_200M_torch.load_checkpoint   
8               timesfm                     TimesFM_2p5_200M_torch.model   
9               timesfm               TimesFM_2p5_200M_torch.push_to_hub   
10              timesfm           TimesFM_2p5_200M_torch.save_pretrained   
16      timesfm.configs                                          Literal   
17      time

In [9]:
api_df

,module,name,kind,signature,doc,file
0,timesfm,ForecastConfig,class,"(max_context: int = 0, max_horizon: int = 0, normalize_inputs: bool = False, window_size: int = 0, per_core_batch_size: int = 1, use_continuous_quantile_head: bool = False, force_flip_invariance: bool = True, infer_is_positive: bool = True, fix_quantile_crossing: bool = False, return_backcast: bool = False) -> None","Options for forecasting.\n\nAttributes:\n max_context: The maximum context length. This is used by the complied decode\n function at inference time during batched inference. Any input time series\n with length less than max_context will be padded with zeros, and with\n length greater than max_context will be truncated.\n max_horizon: The maximum horizon length. This is used by the complied decode\n function at inference time during batched inference. The compiled cached\n decoding function will by default forecast till max_horizon.\n normalize_inputs: Whether to normalize the inputs. This is useful when the\n raw inputs are of extremely large or small magnitudes which may result in\n numerical issues.\n window_size: The window size for decomposed forecasting.\n TODO(siriuz42)",c:\Users\hashimoto.ryohei\miniconda3\envs\kaiseki\Lib\site-packages\timesfm\__init__.py
1,timesfm,TimesFM_2p5_200M_torch,class,"(*args, **kwargs) -> ~T",PyTorch implementation of TimesFM 2.5 with 200M parameters.,c:\Users\hashimoto.ryohei\miniconda3\envs\kaiseki\Lib\site-packages\timesfm\__init__.py
2,timesfm,TimesFM_2p5_200M_torch.compile,method,"(self, forecast_config: timesfm.configs.ForecastConfig, **kwargs) -> None",Attempts to compile the model for fast decoding.\n\nSee configs.ForecastConfig for more details on the supported flags.\n\nArgs:\n forecast_config: Configuration for forecasting flags.\n **kwargs: Additional keyword arguments to pass to model.compile().,c:\Users\hashimoto.ryohei\miniconda3\envs\kaiseki\Lib\site-packages\timesfm\__init__.py
3,timesfm,TimesFM_2p5_200M_torch.forecast,method,"(self, horizon: int, inputs: list[numpy.ndarray]) -> tuple[numpy.ndarray, numpy.ndarray]",Forecasts the time series.,c:\Users\hashimoto.ryohei\miniconda3\envs\kaiseki\Lib\site-packages\timesfm\__init__.py
4,timesfm,TimesFM_2p5_200M_torch.forecast_with_covariates,method,"(self, inputs: list[typing.Sequence[float]], dynamic_numerical_covariates: dict[str, typing.Sequence[typing.Sequence[float]]] | None = None, dynamic_categorical_covariates: dict[str, typing.Sequence[typing.Sequence[int | str]]] | None = None, static_numerical_covariates: dict[str, typing.Sequence[float]] | None = None, static_categorical_covariates: dict[str, typing.Sequence[int | str]] | None = None, xreg_mode: str = 'xreg + timesfm', normalize_xreg_target_per_input: bool = True, ridge: float = 0.0, max_rows_per_col: int = 0, force_on_cpu: bool = False)","Forecasts on a list of time series with covariates.\n\nTo optimize inference speed, avoid string valued categorical covariates.\n\nArgs:\n inputs: A list of time series forecast contexts. Each context time series\n should be in a format convertible to JTensor by `jnp.array`.\n dynamic_numerical_covariates: A dict of dynamic numerical covariates.\n dynamic_categorical_covariates: A dict of dynamic categorical covariates.\n static_numerical_covariates: A dict of static numerical covariates.\n static_categorical_covariates: A dict of static categorical covariates.\n xreg_mode: one of ""xreg + timesfm"" or ""timesfm + xreg"". ""xreg + timesfm""\n fits a model on the residuals of the TimesFM forecast. ""timesfm + xreg""\n fits a model on the targets then forecasts on the residuals via TimesFM.\n norm",c:\Users\hashimoto.ryohei\miniconda3\envs\kaiseki\Lib\site-packages\timesfm\__init__.py
5,timesfm,TimesFM_2p5_200M_torch.from_pretrained,method,"(pretrained_model_name_or_path: Union[str, pathlib.Path], *, force_download: bool = False, resume_download: Optional[bool] = None, proxies: Optional[Dict] = None, token: Union[bool, str, NoneType] = None, cache_dir: Unio

In [10]:
import pandas as pd
import inspect
import timesfm

def get_library_features(module):
    """ライブラリの機能を取得してDataFrameに格納する"""
    
    features = []
    
    # モジュール内のすべての要素を取得
    for name, obj in inspect.getmembers(module):
        # プライベート属性をスキップ
        if name.startswith('_'):
            continue
        
        feature_info = {
            'name': name,
            'type': type(obj).__name__,
            'module': getattr(obj, '__module__', 'N/A'),
        }
        
        # クラスの場合
        if inspect.isclass(obj):
            feature_info['type'] = 'class'
            # メソッド一覧を取得
            methods = [m for m in dir(obj) if not m.startswith('_') and callable(getattr(obj, m, None))]
            feature_info['methods'] = ', '.join(methods[:10])  # 最初の10個のみ
            feature_info['docstring'] = inspect.getdoc(obj) or 'No description'
            
        # 関数の場合
        elif inspect.isfunction(obj) or inspect.isbuiltin(obj):
            feature_info['type'] = 'function'
            try:
                sig = inspect.signature(obj)
                feature_info['signature'] = str(sig)
            except:
                feature_info['signature'] = 'N/A'
            feature_info['docstring'] = inspect.getdoc(obj) or 'No description'
            
        # その他の属性
        else:
            feature_info['docstring'] = str(obj)[:100] if not callable(obj) else 'N/A'
        
        features.append(feature_info)
    
    # DataFrameに変換
    df = pd.DataFrame(features)
    
    return df

# timesfmライブラリの機能を取得
df_features = get_library_features(timesfm)

# 結果を表示
print("=" * 80)
print("TimesFM Library Features")
print("=" * 80)
print(f"\nTotal features found: {len(df_features)}")
print("\n")
print(df_features.to_string())

# カテゴリ別に集計
print("\n" + "=" * 80)
print("Summary by Type")
print("=" * 80)
print(df_features['type'].value_counts())

# クラスだけを抽出
classes_df = df_features[df_features['type'] == 'class'].copy()
if not classes_df.empty:
    print("\n" + "=" * 80)
    print("Classes Details")
    print("=" * 80)
    for idx, row in classes_df.iterrows():
        print(f"\nClass: {row['name']}")
        print(f"Module: {row['module']}")
        print(f"Methods: {row['methods']}")
        print(f"Description: {row['docstring'][:200]}...")

# 関数だけを抽出
functions_df = df_features[df_features['type'] == 'function'].copy()
if not functions_df.empty:
    print("\n" + "=" * 80)
    print("Functions Details")
    print("=" * 80)
    for idx, row in functions_df.iterrows():
        print(f"\nFunction: {row['name']}")
        print(f"Signature: {row.get('signature', 'N/A')}")
        print(f"Description: {row['docstring'][:200] if len(row['docstring']) > 200 else row['docstring']}")

# CSVに保存する場合
df_features.to_csv('timesfm_features.csv', index=False, encoding='utf-8-sig')
print("\n✓ Features saved to 'timesfm_features.csv'")

TimesFM Library Features

Total features found: 7


                     name    type                                 module                                                                                                                                  methods                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [12]:
import pandas as pd
import inspect
import timesfm
from typing import Any, Dict, List

def analyze_module_deeply(module, module_name='timesfm'):
    """モジュールを深く分析してDataFrameに格納"""
    
    all_features = []
    
    def analyze_object(obj, obj_name, parent_name=''):
        """オブジェクトを再帰的に分析"""
        full_name = f"{parent_name}.{obj_name}" if parent_name else obj_name
        
        feature = {
            'full_name': full_name,
            'name': obj_name,
            'parent': parent_name,
            'type': '',
            'is_public': not obj_name.startswith('_'),
            'module': getattr(obj, '__module__', 'N/A'),
            'signature': '',
            'docstring': '',
            'attributes': '',
        }
        
        if inspect.isclass(obj):
            feature['type'] = 'class'
            feature['docstring'] = (inspect.getdoc(obj) or '')[:500]
            
            # クラスのメソッドとプロパティを取得
            methods = []
            properties = []
            for attr_name in dir(obj):
                if attr_name.startswith('_'):
                    continue
                try:
                    attr = getattr(obj, attr_name)
                    if callable(attr):
                        methods.append(attr_name)
                    else:
                        properties.append(attr_name)
                except:
                    pass
            
            feature['attributes'] = f"Methods: {', '.join(methods[:5])}; Properties: {', '.join(properties[:5])}"
            
        elif inspect.isfunction(obj) or inspect.ismethod(obj):
            feature['type'] = 'function'
            try:
                feature['signature'] = str(inspect.signature(obj))
            except:
                feature['signature'] = 'N/A'
            feature['docstring'] = (inspect.getdoc(obj) or '')[:500]
            
        elif inspect.ismodule(obj):
            feature['type'] = 'module'
            feature['docstring'] = (inspect.getdoc(obj) or '')[:500]
            
        else:
            feature['type'] = type(obj).__name__
            feature['docstring'] = str(obj)[:200] if not callable(obj) else ''
        
        return feature
    
    # トップレベルの要素を分析
    for name, obj in inspect.getmembers(module):
        if not name.startswith('_'):
            feature = analyze_object(obj, name)
            all_features.append(feature)
            
            # クラスの場合、メソッドも詳細に分析
            if inspect.isclass(obj) and obj.__module__.startswith(module_name):
                for method_name, method_obj in inspect.getmembers(obj):
                    if not method_name.startswith('_') and (inspect.isfunction(method_obj) or inspect.ismethod(method_obj)):
                        method_feature = analyze_object(method_obj, method_name, name)
                        all_features.append(method_feature)
    
    df = pd.DataFrame(all_features)
    return df

# 詳細分析を実行
df_detailed = analyze_module_deeply(timesfm)

# 公開機能のみをフィルタ
df_public = df_detailed[df_detailed['is_public'] == True].copy()

print("=" * 80)
print("TimesFM Detailed Analysis")
print("=" * 80)
print(f"\nTotal items: {len(df_detailed)}")
print(f"Public items: {len(df_public)}")
print("\n")

# タイプ別の集計
print("Distribution by Type:")
print(df_public['type'].value_counts())

# クラス一覧
print("\n" + "=" * 80)
print("Available Classes:")
print("=" * 80)
classes = df_public[df_public['type'] == 'class']['name'].tolist()
for cls in classes:
    print(f"  - {cls}")

# 関数一覧
print("\n" + "=" * 80)
print("Available Functions:")
print("=" * 80)
top_level_funcs = df_public[(df_public['type'] == 'function') & (df_public['parent'] == '')]['name'].tolist()
for func in top_level_funcs:
    print(f"  - {func}")

# Excel出力する場合
with pd.ExcelWriter('timesfm_analysis.xlsx', engine='openpyxl') as writer:
    df_public.to_excel(writer, sheet_name='All Features', index=False)
    df_public[df_public['type'] == 'class'].to_excel(writer, sheet_name='Classes', index=False)
    df_public[df_public['type'] == 'function'].to_excel(writer, sheet_name='Functions', index=False)

TimesFM Detailed Analysis

Total items: 15
Public items: 15


Distribution by Type:
type
function    8
module      5
class       2
Name: count, dtype: int64

Available Classes:
  - ForecastConfig
  - TimesFM_2p5_200M_torch

Available Functions:


In [16]:
from __future__ import annotations

import json
import re
import subprocess
from dataclasses import dataclass
from pathlib import Path
from typing import Iterable, List, Dict, Tuple, Optional

import pandas as pd


REPO_URL = "https://github.com/google-research/timesfm"
CLONE_DIR = (Path.cwd() / "timesfm_repo").resolve()  # ←絶対パスになります


# =========
# 1) clone
# =========
def ensure_cloned(repo_url: str, clone_dir: Path) -> None:
    if clone_dir.exists() and any(clone_dir.iterdir()):
        return
    clone_dir.parent.mkdir(parents=True, exist_ok=True)
    subprocess.run(
        ["git", "clone", "--depth", "1", repo_url, str(clone_dir)],
        check=True,
    )


# =========================
# 2) ファイルからテキスト抽出
# =========================
TEXT_EXTS = {".py", ".md", ".rst", ".txt", ".sh", ".yml", ".yaml", ".toml", ".ipynb"}

def read_text_from_file(p: Path) -> Optional[str]:
    try:
        if p.suffix == ".ipynb":
            nb = json.loads(p.read_text(encoding="utf-8"))
            chunks = []
            for cell in nb.get("cells", []):
                src = cell.get("source", [])
                if isinstance(src, list):
                    chunks.append("".join(src))
                elif isinstance(src, str):
                    chunks.append(src)
            return "\n".join(chunks)
        else:
            return p.read_text(encoding="utf-8", errors="ignore")
    except Exception:
        return None


# =========================
# 3) 機能カテゴリ（痕跡）定義
# =========================
FEATURE_PATTERNS: Dict[str, List[str]] = {
    "再学習/微調整(fine-tuning)": [
        r"\bfinetune\b", r"fine[-_ ]tune", r"\btrain(ing)?\b",
        r"\blora\b", r"\bdora\b", r"\bpeft\b",
        r"\bpaxml\b", r"\bcheckpoint\b", r"\borbax\b",
        r"\bnum[-_ ]epochs\b", r"\blearning[-_ ]rate\b",
    ],
    "精度評価(metrics/evaluation)": [
        r"\beval\b", r"\bmetric(s)?\b", r"\bbenchmark\b",
        r"\bmae\b", r"\bmse\b", r"\brmse\b", r"\bmape\b", r"\bsmape\b",
        r"\bloss\b", r"\bvalidation\b", r"\bearly[-_ ]stop(ping)?\b",
    ],
    "可視化(visualization)": [
        r"\bmatplotlib\b", r"\bseaborn\b", r"\bplot\b", r"\bchart\b",
        r"\bplt\.", r"\bplotly\b",
    ],
}

@dataclass
class Hit:
    feature: str
    file_path: str
    ext: str
    matches: int
    snippets: List[str]


def find_hits_in_text(text: str, patterns: List[str], max_snips: int = 3) -> Tuple[int, List[str]]:
    total = 0
    snips: List[str] = []
    lines = text.splitlines()

    # 行単位で“どの行が引っかかったか”を拾う（初心者でも追跡しやすい）
    for i, line in enumerate(lines):
        for pat in patterns:
            if re.search(pat, line, flags=re.IGNORECASE):
                total += 1
                if len(snips) < max_snips:
                    # 周辺行も少し付ける
                    ctx = "\n".join(lines[max(0, i-1): min(len(lines), i+2)])
                    snips.append(ctx)
                break
    return total, snips


# =========================
# 4) 走査 → df化
# =========================
def scan_repo_to_df(repo_dir: Path) -> pd.DataFrame:
    hits: List[Hit] = []

    for p in repo_dir.rglob("*"):
        if not p.is_file():
            continue
        if p.suffix.lower() not in TEXT_EXTS:
            continue

        text = read_text_from_file(p)
        if not text:
            continue

        for feat, pats in FEATURE_PATTERNS.items():
            n, snips = find_hits_in_text(text, pats)
            if n > 0:
                hits.append(
                    Hit(
                        feature=feat,
                        file_path=str(p.resolve()),
                        ext=p.suffix.lower(),
                        matches=n,
                        snippets=snips,
                    )
                )

    df = pd.DataFrame([h.__dict__ for h in hits])
    if df.empty:
        return df

    # 見やすく：強い痕跡（matchesが多い）順
    df = df.sort_values(["feature", "matches"], ascending=[True, False]).reset_index(drop=True)

    # snippetsは長いのでJSON文字列化（Excelに落としても崩れにくい）
    df["snippets"] = df["snippets"].apply(lambda xs: json.dumps(xs, ensure_ascii=False, indent=2))
    return df


def main() -> None:
    ensure_cloned(REPO_URL, CLONE_DIR)
    df = scan_repo_to_df(CLONE_DIR)

    out_xlsx = (Path.cwd() / "timesfm_repo_feature_hits.xlsx").resolve()
    with pd.ExcelWriter(out_xlsx, engine="openpyxl") as w:
        df.to_excel(w, index=False, sheet_name="hits")

    print("Repo:", CLONE_DIR)
    print("Rows:", len(df))
    print("Saved:", out_xlsx)


if __name__ == "__main__":
    main()


Repo: C:\model_info\timesfm_repo
Rows: 56
Saved: C:\model_info\timesfm_repo_feature_hits.xlsx


In [17]:
import ast
import os
import pandas as pd
from typing import List, Dict, Any, Optional

def get_annotation_str(annotation: Optional[ast.AST]) -> str:
    """型アノテーションを文字列に変換するヘルパー関数"""
    if annotation is None:
        return "Any"
    try:
        if hasattr(ast, 'unparse'):
            return ast.unparse(annotation)
        else:
            # Python 3.8以下用（簡易版）
            if isinstance(annotation, ast.Name):
                return annotation.id
            return "ComplexType"
    except:
        return "Unknown"

def get_value_str(value_node: Optional[ast.AST]) -> str:
    """デフォルト値を文字列に変換するヘルパー関数"""
    if value_node is None:
        return "-"
    try:
        if hasattr(ast, 'unparse'):
            return ast.unparse(value_node)
        # リテラル値の処理
        if isinstance(value_node, (ast.Constant, ast.Str, ast.Num)):
            return str(value_node.value) if hasattr(value_node, 'value') else str(value_node.s)
        return "ComplexValue"
    except:
        return "Error"

def analyze_file(file_path: str, target_classes: List[str] = None) -> List[Dict[str, Any]]:
    """指定されたファイルを解析し、クラス・メソッド・引数の情報を抽出する"""
    if not os.path.exists(file_path):
        print(f"Warning: File not found: {file_path}")
        return []

    with open(file_path, "r", encoding="utf-8") as f:
        try:
            tree = ast.parse(f.read())
        except Exception as e:
            print(f"Error parsing {file_path}: {e}")
            return []

    results = []
    
    for node in ast.walk(tree):
        if isinstance(node, ast.ClassDef):
            class_name = node.name
            if target_classes and class_name not in target_classes:
                continue

            # Docstringの取得 (クラス)
            class_doc = ast.get_docstring(node)
            
            # 1. Dataclassフィールドの解析 (Configs用)
            # クラス直下のAnnotated Assignment (例: max_context: int = 512) を探す
            for item in node.body:
                if isinstance(item, ast.AnnAssign) and isinstance(item.target, ast.Name):
                    field_name = item.target.id
                    field_type = get_annotation_str(item.annotation)
                    default_val = get_value_str(item.value)
                    
                    results.append({
                        "Category": "Configuration",
                        "Class": class_name,
                        "Method/Field": "(Config Field)",
                        "Argument": field_name,
                        "Type": field_type,
                        "Required": "No" if item.value else "Yes",
                        "Default": default_val,
                        "Description": "See Class Docstring" 
                    })

            # 2. メソッドの解析
            for item in node.body:
                if isinstance(item, ast.FunctionDef):
                    method_name = item.name
                    if method_name.startswith("_") and method_name != "__init__":
                        continue # プライベートメソッドは除外

                    method_doc = ast.get_docstring(item)
                    
                    # 引数のデフォルト値処理
                    # defaultsは後ろの引数から対応するため、位置合わせが必要
                    num_defaults = len(item.args.defaults)
                    num_args = len(item.args.args)
                    args_with_defaults = item.args.args[num_args - num_defaults:]
                    defaults_map = {}
                    for arg, default in zip(args_with_defaults, item.args.defaults):
                        defaults_map[arg.arg] = get_value_str(default)

                    # 全引数をスキャン
                    for arg in item.args.args:
                        arg_name = arg.arg
                        if arg_name == "self":
                            continue
                        
                        arg_type = get_annotation_str(arg.annotation)
                        is_required = arg_name not in defaults_map
                        default_val = defaults_map.get(arg_name, "-")

                        results.append({
                            "Category": "Method Argument",
                            "Class": class_name,
                            "Method/Field": method_name,
                            "Argument": arg_name,
                            "Type": arg_type,
                            "Required": "Yes" if is_required else "No",
                            "Default": default_val,
                            "Description": (method_doc.split('\n')[0] if method_doc else "")[:100]
                        })

    return results

def main():
    # 解析対象のファイルパス定義
    # ※ 環境に合わせてパスを修正してください
    target_files = {
        "Configs": ("src/timesfm/configs.py", ["ForecastConfig"]),
        "TorchModel": ("src/timesfm/timesfm_2p5/timesfm_2p5_torch.py", ["TimesFM_2p5_200M_torch"]),
        "BaseModel": ("src/timesfm/timesfm_2p5/timesfm_2p5_base.py", ["TimesFM_2p5_200M_torch", "TimesFM_2p5Base"]),
    }

    all_data = []
    print("解析を開始します...")

    for label, (path, classes) in target_files.items():
        print(f"Processing {label}: {path}")
        data = analyze_file(path, target_classes=classes)
        all_data.extend(data)

    if not all_data:
        print("データが見つかりませんでした。パスを確認してください。")
        return

    # DataFrame作成
    df = pd.DataFrame(all_data)
    
    # 見やすい順序に並べ替え
    sort_order = ["Configuration", "Method Argument"]
    df["Category"] = pd.Categorical(df["Category"], categories=sort_order, ordered=True)
    df = df.sort_values(by=["Category", "Class", "Method/Field"]).reset_index(drop=True)

    # 結果の表示
    print("\n=== TimesFM Execution Parameters ===")
    print(df.to_markdown(index=False))
    
    # CSV出力
    df.to_csv("timesfm_execution_table.csv", index=False)
    print("\n'timesfm_execution_table.csv' に保存しました。")

if __name__ == "__main__":
    main()

解析を開始します...
Processing Configs: src/timesfm/configs.py
Processing TorchModel: src/timesfm/timesfm_2p5/timesfm_2p5_torch.py
Processing BaseModel: src/timesfm/timesfm_2p5/timesfm_2p5_base.py
データが見つかりませんでした。パスを確認してください。
